In [23]:
import cv2
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline  

In [65]:
rotation_range = 10
zoom_range = 0.05
shift_range = 0.05

In [66]:
image = cv2.imread("./cage.jpg", cv2.COLOR_RGB2BGR).astype(np.float32)
image = cv2.resize(image, (256, 256))

In [67]:
h, w = image.shape[0:2]

In [68]:
rotation = np.random.uniform(-rotation_range, rotation_range)
scale = np.random.uniform(1 - zoom_range, 1 + zoom_range)
tx = np.random.uniform(-shift_range, shift_range) * w
ty = np.random.uniform(-shift_range, shift_range) * h

In [69]:
mat = cv2.getRotationMatrix2D((w // 2, h // 2), rotation, scale)
mat[:, 2] += (tx, ty)
result = cv2.warpAffine(image, mat, (w, h), borderMode=cv2.BORDER_REPLICATE)

In [70]:
#result = result.transpose((1, 2, 0))
cv2.imwrite("./cage2.jpg", result)

True